# Data Engineering

## Import Library

In [1]:
import pandas as pd
import numpy as np

## Read In Raw Data

In [2]:
OES = pd.read_csv("~/Desktop/WhatDataScience/OES1999_2016.csv")
inflation_usa = pd.read_csv("~/Desktop/WhatDataScience/inflation_usa.csv")

In [3]:
OES = OES[OES['year']>2002]

In [4]:
OES['a_mean'] = OES['a_mean'].apply(int)
OES['tot_emp'] = OES['tot_emp'].apply(int)
OES['sum_mean'] = OES['a_mean']*OES['tot_emp']

## Overall Trend

In [5]:
OES[0:5]

,st,state,a_mean,tot_emp,a_median,occ_title,group,year,sum_mean
0,AL,Alabama,71920,91920,62960,Management occupations,major,2003,6610886400
1,AL,Alabama,50310,54010,45670,Business and financial operations occupations,major,2003,2717243100
2,AL,Alabama,59440,29600,56170,Computer and mathematical occupations,major,2003,1759424000
3,AL,Alabama,57670,35490,55060,Architecture and engineering occupations,major,2003,2046708300
4,AL,Alabama,46870,10800,41630,"Life, physical, and social science occupations",major,2003,506196000


In [6]:
OES_overall_by_state = OES.groupby(['st','year'])['sum_mean','tot_emp'].sum().reset_index()
OES_overall_by_state['a_mean'] = np.round(OES_overall_by_state['sum_mean']/OES_overall_by_state['tot_emp'],0)
OES_overall = OES.groupby(['year'])['sum_mean','tot_emp'].sum().reset_index()
OES_overall['a_mean'] = np.round(OES_overall['sum_mean']/OES_overall['tot_emp'],0)

In [7]:
OES_overall

,year,sum_mean,tot_emp,a_mean
0,2003,4633416585300,128498410,36058.0
1,2004,4763276593600,129170580,36876.0
2,2005,4787433271400,126764390,37766.0
3,2006,5221810918300,133687900,39060.0
4,2007,5496191147500,135406330,40590.0
5,2008,5694387573900,135718090,41957.0
6,2009,5710024087800,131684980,43361.0
7,2010,5664495433800,127893340,44291.0
8,2011,5829242391400,129301640,45083.0
9,2012,5989334589200,131215960,45645.0


## Filter by Occupation Type - Data Science Related

In [8]:
OES['occ_title'] = OES['occ_title'].apply(lambda x:x.lower())

In [9]:
occ_title = list(OES['occ_title'].unique())

In [10]:
occ_title

['management occupations',
 'business and financial operations occupations',
 'computer and mathematical occupations',
 'architecture and engineering occupations',
 'life, physical, and social science occupations',
 'community and social services occupations',
 'legal occupations',
 'education, training, and library occupations',
 'arts, design, entertainment, sports, and media occupations',
 'healthcare practitioners and technical occupations',
 'healthcare support occupations',
 'protective service occupations',
 'food preparation and serving related occupations',
 'building and grounds cleaning and maintenance occupations',
 'personal care and service occupations',
 'sales and related occupations',
 'office and administrative support occupations',
 'farming, fishing, and forestry occupations',
 'construction and extraction occupations',
 'installation, maintenance, and repair occupations',
 'production occupations',
 'transportation and material moving occupations',
 'computer and m

In [11]:
occ_cm = ['computer and mathematical science occupations','computer and mathematical occupations']

In [12]:
OES['filter'] = OES['occ_title'].apply(lambda x: True if x in occ_cm else False)

In [13]:
OESDataScience = OES[OES['filter']]

In [14]:
OESDataScience.drop(['filter','occ_title'],axis=1,inplace=True)

/Users/rinalin/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## Data Science Trend

In [15]:
OESDS_agg = OESDataScience[['st','tot_emp','year','sum_mean']].groupby(['st','year']).sum().reset_index()

In [17]:
OESDS_agg['a_mean'] = OESDS_agg.apply(lambda x:int(x['sum_mean']/x['tot_emp']),axis=1)

In [18]:
OESDS_agg[0:5]

,st,year,tot_emp,sum_mean,a_mean
0,AK,2003,3270,183774000,56200
1,AK,2004,3500,203490000,58140
2,AK,2005,3950,235973000,59740
3,AK,2006,3930,243581400,61980
4,AK,2007,3850,247516500,64290


In [ ]:
inflation_usa

## Merge with Inflation Data

### 1). With Overall Data

In [19]:
OES_overall_by_state = OES_overall_by_state.merge(inflation_usa,how='left',right_on='year',left_on='year')
OES_overall_by_state['a_mean_inflation_adjusted'] = OES_overall_by_state.apply(lambda x: int(x['a_mean']*x['Inflation Rate']),axis=1)
OES_overall = OES_overall.merge(inflation_usa,how='left',right_on='year',left_on='year')
OES_overall['a_mean_inflation_adjusted'] = OES_overall.apply(lambda x: int(x['a_mean']*x['Inflation Rate']),axis=1)

### 2). With Data Science Data

In [20]:
OESDS_complete = OESDS_agg.merge(inflation_usa,how='left',right_on='year',left_on='year')
OESDS_complete['a_mean_inflation_adjusted'] = OESDS_complete.apply(lambda x: int(x['a_mean']*x['Inflation Rate']),axis=1)

In [21]:
OES_overall[0:5]

,year,sum_mean,tot_emp,a_mean,Inflation Rate,a_mean_inflation_adjusted
0,2003,4633416585300,128498410,36058.0,1.303792,47012
1,2004,4763276593600,129170580,36876.0,1.293965,47716
2,2005,4787433271400,126764390,37766.0,1.269071,47927
3,2006,5221810918300,133687900,39060.0,1.236172,48284
4,2007,5496191147500,135406330,40590.0,1.199326,48680


In [22]:
OESDS_complete[0:5]

,st,year,tot_emp,sum_mean,a_mean,Inflation Rate,a_mean_inflation_adjusted
0,AK,2003,3270,183774000,56200,1.303792,73273
1,AK,2004,3500,203490000,58140,1.293965,75231
2,AK,2005,3950,235973000,59740,1.269071,75814
3,AK,2006,3930,243581400,61980,1.236172,76617
4,AK,2007,3850,247516500,64290,1.199326,77104


## Save Result

In [23]:
OESDS_complete.to_csv("~/Desktop/WhatDataScience/computer_math2003_2016.csv",index=False)

In [24]:
OES_overall_by_state.to_csv("~/Desktop/WhatDataScience/all2003_2016.csv",index=False)